# Chain of Thought Reasoning

This is the kind of prompting technique where LLMs are made to reason in steps to reach to a final response or answer. 

We will first import required libraries and load the API key.

In [ ]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']

We create a helper function which is useful to get the completion of the input prompt using an LLM (GPT-3.5-turbo in this case).

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

We define the system message containing steps that it needs to follow and all the data from which it can get the required information to generate a response. One can think of this as our database of movies that we enter in the system prompt to make LLM aware of this data.

In [ ]:
delimiter = "####"
system_message = f"""
You are an assistant who provides information about movies.\
Follow these steps to answer the viewer queries.\
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific movie or genre. \ 

Step 2:{delimiter} If the user is asking about \
specific movie or genre, identify whether \
the movie is present in the following list.

All available movies: 
Movie 1:
Title: "The Secret Life of Dreams"
Genre: Fantasy, Drama
Rating: 8.2/10
Cast: Emma Thompson, Tom Hanks, Lily James
Release Date: July 15, 2022

Movie 2:
Title: "Echoes of Eternity"
Genre: Mystery, Thriller
Rating: 7.6/10
Cast: Mark Ruffalo, Emily Blunt, Michael Fassbender
Release Date: November 3, 2021

Movie 3:
Title: "Rhythm of Redemption"
Genre: Musical, Romance
Rating: 6.9/10
Cast: Anna Kendrick, Chris Evans, Zendaya
Release Date: February 28, 2023

Movie 4:
Title: "The Edge of Darkness"
Genre: Action, Sci-Fi
Rating: 7.8/10
Cast: Scarlett Johansson, Chris Pratt, Robert Downey Jr.
Release Date: September 10, 2022

Step 3:{delimiter} If the message contains movie(s) \
in the list above, retrieve the information for those movies in particular.\

Step 4:{delimiter}: Respond with the retrieved information in an easy to understand\
precise manner.
Answer the viewer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

Once the steps are defined in system message, user query can be passed through LLMs along with the system message to get a reponse. The observed reponse is as expected and all the steps(reason) are given as to how the answer was reached.

In [ ]:
user_message = f"""

I want to know about a movie its name was something like heros of eternity, \
also tell me about any sci-fi movie

"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking about two different movies, one with a specific name and the other in a specific genre.

Step 2:#### The movie "Echoes of Eternity" is present in the list of available movies. There is also a sci-fi movie called "The Edge of Darkness" in the list.

Step 3:#### "Echoes of Eternity" is a mystery thriller movie with a rating of 7.6/10. It stars Mark Ruffalo, Emily Blunt, and Michael Fassbender. The release date of the movie is November 3, 2021. "The Edge of Darkness" is an action sci-fi movie with a rating of 7.8/10. It stars Scarlett Johansson, Chris Pratt, and Robert Downey Jr. The release date of the movie is September 10, 2022.

Response to user:#### "Echoes of Eternity" is a mystery thriller movie with Mark Ruffalo, Emily Blunt, and Michael Fassbender. It was released on November 3, 2021. "The Edge of Darkness" is an action sci-fi movie with Scarlett Johansson, Chris Pratt, and Robert Downey Jr. It will be released on September 10, 2022.

## Few comments on chain of thought

This is one of example of CoT (Chain of Thought) prompting. In other cases of CoT rather than mentioning steps, one can also add "Lets think step by step" or similar instructions to the prompt so that LLMs lists down its thoughts step by step and try to reason to reach the answer.

One major drawback of such case is when our database is huge (which is typically the case). We cannot list down the complete database in prompt itself in that case. This is due to the token limitations of the prompt that can be given and also the cost involved per token to get the response. To resolve this, we can use something called as __Chaining Prompts__. This is discussed in another notebook.